# Explorando bases vetoriais

In [1]:
from langchain.document_loaders import PyPDFLoader

pdf_path = 'data/PPC_TSI_EaD.pdf'
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
)

texts = text_splitter.split_documents(pages)

print(texts[0])

page_content='Ministério da Educação
Instituto Federal de Educação, Ciência e Tecnologia do Rio Grande do Norte
DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN DELIBERAÇÃO Nº 59/2022 - CONSEPEX/IFRN' metadata={'source': 'data/PPC_TSI_EaD.pdf', 'page': 0}


In [3]:
print (f"Quantidade de chunks: {len(texts)} ")

Quantidade de chunks: 3183 


In [5]:
!pip install faiss-cpu
#pip install faiss-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 6.8 MB/s eta 0:00:00a 0:00:01


In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.from_documents(texts,  OllamaEmbeddings(model="mxbai-embed-large"))
query = "Quais são as disciplinas do curso?"
docs = db.similarity_search(query)
print(docs[0].page_content)

disciplinas do curso.


In [7]:
# Perform similarity search to get the top 5 most relevant chunks (by similarity)
docs = db.similarity_search(query, k=5)

# Print the content of each of the top 5 chunks
for i, doc in enumerate(docs):
    print(f"Chunk {i + 1}: {doc.page_content}")

Chunk 1: disciplinas do curso.
Chunk 2: funcionamento do curso por disciplina, estão expostos em tabelas e apresentados nos  
Quadro 17 e 18 (Anexo A).
Chunk 3: disciplina. 
 Os aspectos operacionais relativos ao aproveitamento de estudos e à certificação d e 
conhecimentos, adquiridos através de experiências vivenciadas previamente ao iní cio do curso,
Chunk 4: necessários ao funcionamento do Curso, tomando por base o desenvolvimento simultâneo de 
uma turma para cada período do curso, correspondente ao Quadro 1.
Chunk 5: a) Coordenador de Curso, como Presidente;  
b) todos os docentes em exercício nas atividades de ensino, de pesquisa ou de extensão no Curso 
ou que tenham atuado nos dois últimos semestres letivos;
